## IMPORTS and UTILS

In [3]:
import os
import pandas as pd
import numpy as np
from scipy.stats import entropy
from math import sqrt
import seaborn as sns

import matplotlib.pyplot as plt

from joblib import Parallel, delayed


from robust_evaluation_tools.robust_utils import get_complete_combination, get_metrics, get_diseases
from robust_evaluation_tools.robust_outlier_detection import find_outliers, analyze_detection_performance
from robust_evaluation_tools.synthectic_sites_generations import generate_sites
from robust_evaluation_tools.robust_analysis import calculate_precision_by_bundle
from clinical_combat.harmonization.QuickCombat import QuickCombat

all_metrics = get_metrics()
all_diseases = get_diseases(True)

MAINFOLDER = "RESULTS/DISTRIBUTION_ANALYSIS"
SYNTHETIC_SITES = f"{MAINFOLDER}/SYNTHETIC_SITES"

ANALYSIS_FOLDER = "ANALYSIS"

raw_directory = os.path.join('DONNES', 'COMPILATIONS')

In [ ]:
# Fonction pour calculer la divergence KL
def kl_divergence(p, q):
    return entropy(p, q)

## EXECUTOR

In [ ]:
# AFFICHE LES SCATTER PLOTS AU LIEU DES DISTRIBUZTIONS

# import seaborn as sns
# import matplotlib.pyplot as plt

# # Assuming 'HC' is a specific value in the 'disease' column that indicates healthy controls
# HC_LABEL = 'HC'

# # Plot distribution for each bundle
# for disease in all_diseases:
#     combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))
#     metric_bundles = combination['metric_bundle'].unique()
#     for metric_bundle in metric_bundles:
#         subset_all = combination[combination['metric_bundle'] == metric_bundle]
#         subset = subset_all[subset_all['old_site'] != 'CamCAN']
#         bundle = subset_all.bundle.unique()[0]
#         metric = subset_all.metric.unique()[0]
#         camCan_subset = subset_all[subset_all['old_site'] == 'CamCAN']
        
#         # Scatter plot with mean
#         plt.figure(figsize=(10, 6))
#         plt.scatter(subset[subset['disease'] == HC_LABEL]['age'], subset[subset['disease'] == HC_LABEL]['mean'], color='blue', label='HC', alpha=0.5)
#         plt.scatter(camCan_subset['age'], camCan_subset['mean'], color='green', label='CamCAN', alpha=0.5)
#         plt.scatter(subset[subset['disease'] != HC_LABEL]['age'], subset[subset['disease'] != HC_LABEL]['mean'], color='red', label=f'{disease}', alpha=0.5)
        
#         plt.title(f'Distribution of {disease} in metric {metric} for bundle: {bundle} (mean)')
#         plt.xlabel('Age')
#         plt.ylabel('Mean')
#         plt.legend()
#         plt.show()
        
#         # Scatter plot with mean_no_cov
#         plt.figure(figsize=(10, 6))
#         plt.scatter(subset[subset['disease'] == HC_LABEL]['age'], subset[subset['disease'] == HC_LABEL]['mean_no_cov'], color='blue', label='HC', alpha=0.5)
#         plt.scatter(camCan_subset['age'], camCan_subset['mean_no_cov'], color='green', label='CamCAN', alpha=0.5)
#         plt.scatter(subset[subset['disease'] != HC_LABEL]['age'], subset[subset['disease'] != HC_LABEL]['mean_no_cov'], color='red', label=f'{disease}', alpha=0.5)
        
#         plt.title(f'Distribution of {disease} in metric {metric} for bundle: {bundle} (mean_no_cov)')
#         plt.xlabel('Age')
#         plt.ylabel('Mean_no_cov')
#         plt.legend()
#         plt.show()

In [ ]:

# Assuming 'HC' is a specific value in the 'disease' column that indicates healthy controls
HC_LABEL = 'HC'
DISTRIBUTION_FOLDER = os.path.join(MAINFOLDER, 'COMPILATION_DISTRIBUTION')
os.makedirs(DISTRIBUTION_FOLDER, exist_ok=True)

def process_disease(disease, raw_directory, DISTRIBUTION_FOLDER):
    print(f"Processing {disease}...")  # Debugging info
    combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))
    metric_bundles = combination['metric_bundle'].unique()
    
    for metric_bundle in metric_bundles:
        subset_all = combination[combination['metric_bundle'] == metric_bundle]
        subset = subset_all[subset_all['old_site'] != 'CamCAN']
        
        bundle = subset_all.bundle.unique()[0]
        metric = subset_all.metric.unique()[0]
        camCan_subset = subset_all[subset_all['old_site'] == 'CamCAN']

        plt.figure(figsize=(10, 6))
        sns.kdeplot(subset[subset['disease'] == HC_LABEL]['mean_no_cov'], color='blue', label='HC', fill=True)
        sns.kdeplot(camCan_subset['mean_no_cov'], color='green', label='CamCAN', fill=True)
        sns.kdeplot(subset[subset['disease'] != HC_LABEL]['mean_no_cov'], color='red', label=f'{disease}', fill=True)
        plt.title(f'Distribution of {disease} in metric {metric} for bundle: {bundle}')
        plt.xlabel('mean_no_cov')
        plt.ylabel('Frequency')
        plt.legend()

        output_dir = os.path.join(DISTRIBUTION_FOLDER, disease, metric)
        os.makedirs(output_dir, exist_ok=True)

        # Save the plot
        output_path = os.path.join(output_dir, f'{metric_bundle}.png')
        plt.savefig(output_path)
        plt.close()

# Run diseases in parallel, but keep metric bundles sequential
Parallel(n_jobs=-1)(
    delayed(process_disease)(disease, raw_directory, DISTRIBUTION_FOLDER)
    for disease in all_diseases
)

In [ ]:

# Assuming 'HC' is a specific value in the 'disease' column that indicates healthy controls
HC_LABEL = 'HC'

# Function to process one disease
def process_disease(disease, raw_directory):
    print(f"Processing {disease}...")  # Debugging info
    results = []
    
    # Load the data for the disease
    combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))
    metric_bundles = combination['metric_bundle'].unique()
    
    for metric_bundle in metric_bundles:
        subset_all = combination[combination['metric_bundle'] == metric_bundle]
        bundle = subset_all.bundle.unique()[0]
        metric = subset_all.metric.unique()[0]

        # Subsets for HC and non-HC
        hc_data = subset_all[subset_all['disease'] == HC_LABEL]['mean_no_cov'].dropna()
        non_hc_data = subset_all[subset_all['disease'] != HC_LABEL]['mean_no_cov'].dropna()

        # Ensure both distributions are not empty
        if len(hc_data) > 1 and len(non_hc_data) > 1:
            # Create histograms to normalize distributions
            hc_hist, bins = np.histogram(hc_data, bins=50, density=True)
            non_hc_hist, _ = np.histogram(non_hc_data, bins=bins, density=True)

            # Compute metrics
            mean_hc = np.mean(hc_data)
            mean_non_hc = np.mean(non_hc_data)
            std_hc = np.std(hc_data)
            std_non_hc = np.std(non_hc_data)
            var_hc = np.var(hc_data, ddof=1)
            var_non_hc = np.var(non_hc_data, ddof=1)
            n_hc = len(hc_data)
            n_non_hc = len(non_hc_data)

            # Pooled standard deviation
            pooled_std = abs(np.sqrt(((n_hc - 1) * var_hc + (n_non_hc - 1) * var_non_hc) / (n_hc + n_non_hc - 2)))

            # Compute distance metrics
            bhatt_dist = QuickCombat.bhattacharyya_distance(hc_data, non_hc_data)
            kl_div = kl_divergence(hc_hist + 1e-10, non_hc_hist + 1e-10)  # Add epsilon to avoid division by zero
            euclidean_dist = abs(mean_hc - mean_non_hc)
            mahalanobis_dist = abs(mean_hc - mean_non_hc) / sqrt((std_hc**2 + std_non_hc**2) / 2)
            d_cohen = abs(mean_hc - mean_non_hc) / pooled_std

            # Store results
            results.append({
                'disease': disease,
                'metric_bundle': metric_bundle,
                'bundle': bundle,
                'metric': metric,
                'bhattacharyya_distance': bhatt_dist,
                'kl_divergence': kl_div,
                'euclidean_distance': euclidean_dist,
                'mahalanobis_distance': mahalanobis_dist,
                'prct_distance': abs(euclidean_dist / mean_hc),
                'd_cohen': d_cohen
            })
    
    return results


# Run diseases in parallel, but keep metric bundles sequential
all_results = Parallel(n_jobs=-1)(
    delayed(process_disease)(disease, raw_directory)
    for disease in all_diseases
)

# Flatten results
results = [item for sublist in all_results for item in sublist]

# Convert results into DataFrame
results_df = pd.DataFrame(results)

# Save results
output_dir = MAINFOLDER
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, 'distance_metrics_results.csv')
results_df.to_csv(output_path, index=False)

print(f"Results saved to {output_path}")


In [ ]:
# Initialisation du DataFrame de résultat pour les valeurs maximales
max_results = []

output_path = f'{MAINFOLDER}/distance_metrics_results.csv'
results_df = pd.read_csv(output_path)

# Parcourir chaque maladie
for disease in results_df['disease'].unique():
    disease_subset = results_df[results_df['disease'] == disease]
    
    # Identifier les metric_bundles avec les valeurs maximales pour chaque mesure de distance
    max_bhatt = disease_subset.loc[disease_subset['bhattacharyya_distance'].idxmax()]
    max_kl = disease_subset.loc[disease_subset['kl_divergence'].idxmax()]
    max_euclidean = disease_subset.loc[disease_subset['euclidean_distance'].idxmax()]
    max_mahalanobis = disease_subset.loc[disease_subset['mahalanobis_distance'].idxmax()]
    
    # Ajouter les résultats dans une liste
    max_results.append({
        'disease': disease,
        'metric_bundle_max_bhatt': max_bhatt['metric_bundle'],
        'bhattacharyya_distance': max_bhatt['bhattacharyya_distance'],
        'metric_bundle_max_kl': max_kl['metric_bundle'],
        'kl_divergence': max_kl['kl_divergence'],
        'metric_bundle_max_euclidean': max_euclidean['metric_bundle'],
        'euclidean_distance': max_euclidean['euclidean_distance'],
        'metric_bundle_max_mahalanobis': max_mahalanobis['metric_bundle'],
        'mahalanobis_distance': max_mahalanobis['mahalanobis_distance']
    })

# Convertir les résultats en DataFrame
max_results_df = pd.DataFrame(max_results)

# Afficher les résultats
print(max_results_df)

In [ ]:
results_df
results_df = results_df[~results_df['bundle'].isin(['left_ventricle', 'right_ventricle'])]


In [ ]:
# Define the output directory
column = 'prct_distance'

output_dir = f'{MAINFOLDER}/PRCT_HISTOGRAMS/{column}/PER_METRIC'
os.makedirs(output_dir, exist_ok=True)

# Parcourir chaque maladie
for disease in results_df['disease'].unique():
    disease_subset = results_df[results_df['disease'] == disease]
    
    # Parcourir chaque metric
    for metric in disease_subset['metric'].unique():
        metric_subset = disease_subset[disease_subset['metric'] == metric]
        
        # Afficher l'histogramme de la distribution de prct_distance
        plt.figure(figsize=(10, 6))
        plt.hist(metric_subset[column], bins=30, alpha=0.7, color='blue', edgecolor='black')
        plt.title(f'Distribution of {column} for {disease} - {metric}')
        plt.xlabel(f'{column}')
        plt.ylabel('Frequency')
        plt.grid(True)
        
        # Create the directory for the disease and metric
        disease_dir = os.path.join(output_dir, disease)
        os.makedirs(disease_dir, exist_ok=True)
        
        # Save the plot
        output_path = os.path.join(disease_dir, f'{disease}_{metric}_{column}_histogram.png')
        plt.savefig(output_path)
        plt.close()# Define the output directory

In [ ]:
# Define the output directory
output_dir = f'{MAINFOLDER}/PRCT_HISTOGRAMS/{column}/PER_DISEASE'
os.makedirs(output_dir, exist_ok=True)

# Parcourir chaque maladie
for disease in results_df['disease'].unique():
    disease_subset = results_df[results_df['disease'] == disease]  
    # Afficher l'histogramme de la distribution de prct_distance
    plt.figure(figsize=(10, 6))
    plt.hist(disease_subset[column], bins=30, alpha=0.7, color='blue', edgecolor='black')
    plt.title(f'Distribution of {column} for {disease} -')
    plt.xlabel(f'{column}')
    plt.ylabel('Frequency')
    plt.grid(True)
    
    # Save the plot
    output_path = os.path.join(output_dir, f'{disease}__{column}_histogram.png')
    plt.savefig(output_path)
    plt.close()

In [ ]:
output_dir = f'{MAINFOLDER}/PRCT_HISTOGRAMS/{column}/PER_METRIC_ALL'
os.makedirs(output_dir, exist_ok=True)

# Récupérer la liste unique des metrics
all_metrics = results_df['metric'].unique()

# Boucle sur chaque metric
for metric in all_metrics:
    # Filtrer le DataFrame sur le metric courant
    metric_subset = results_df[results_df['metric'] == metric]
    
    # Créer une figure
    plt.figure(figsize=(10, 6))
    
    # Récupérer la liste unique des maladies pour ce metric
    all_diseases = metric_subset['disease'].unique()
    
    # Pour chaque maladie, tracer l’histogramme de prct_distance
    for disease in all_diseases:
        disease_subset = metric_subset[metric_subset['disease'] == disease]
        plt.hist(
            disease_subset[column], 
            bins=30, 
            alpha=0.5, 
            label=disease, 
            edgecolor='black'
        )
    
    # Ajouter légende et titres
    plt.title(f'Distribution de prct_distance pour le metric : {metric}')
    plt.xlabel(f'{column}')
    plt.ylabel('Fréquence')
    plt.legend()
    plt.grid(True)

    # Enregistrer la figure
    output_path = os.path.join(output_dir, f'{metric}_all_diseases_{column}_histogram.png')
    plt.savefig(output_path)
    plt.close()

In [ ]:
output_dir = f'{MAINFOLDER}/PRCT_HISTOGRAMS/{column}'
os.makedirs(output_dir, exist_ok=True)


# Boucle sur chaque metric
    
# Créer une figure
plt.figure(figsize=(10, 6))

# Récupérer la liste unique des maladies pour ce metric
all_diseases = results_df['disease'].unique()

# Pour chaque maladie, tracer l’histogramme de prct_distance
for disease in all_diseases:
    disease_subset = results_df[results_df['disease'] == disease]
    plt.hist(
        disease_subset[column], 
        bins=30, 
        alpha=0.5, 
        label=disease, 
        edgecolor='black'
    )

# Ajouter légende et titres
plt.title(f'Distribution de {column} pour le metric : {metric}')
plt.xlabel('prct_distance')
plt.ylabel('Fréquence')
plt.legend()
plt.grid(True)

# Enregistrer la figure
output_path = os.path.join(output_dir, f'all_diseases_{column}_histogram.png')
plt.savefig(output_path)
plt.close()

In [ ]:
# Create subset of diseases where the string starts with 'SYN'
syn_diseases = ["SYN_0.5", "SYN_1",]
output_dir = f'{MAINFOLDER}/PRCT_HISTOGRAMS/{column}/PER_METRIC_WITH_SYN'
os.makedirs(output_dir, exist_ok=True)

# Parcourir chaque maladie
for disease in all_diseases:
    disease_subset = results_df[results_df['disease'] == disease]
    
    # Parcourir chaque metric
    for metric in disease_subset['metric'].unique():
        metric_subset = disease_subset[disease_subset['metric'] == metric]
        
        # Afficher l'histogramme de la distribution de prct_distance
        plt.figure(figsize=(10, 6))
        for dis in syn_diseases:
            syn_sub = results_df[results_df['disease'] == dis]
            syn_met = syn_sub[syn_sub['metric'] == metric]
            plt.hist(
                syn_met[column], 
                bins=30, 
                alpha=0.2, 
                label=dis, 
                edgecolor='black'
            )
        plt.hist(metric_subset[column], bins=30, alpha=0.7, color='blue', edgecolor='black', label=disease)
        plt.title(f'Distribution of {column} for {disease} - {metric}')
        plt.xlabel(f'{column}')
        plt.ylabel('Frequency')
        plt.legend()
        plt.grid(True)

        # Pour chaque maladie, tracer l’histogramme de prct_distance
        
        # Create the directory for the disease and metric
        disease_dir = os.path.join(output_dir, disease)
        os.makedirs(disease_dir, exist_ok=True)
        
        # Save the plot
        output_path = os.path.join(disease_dir, f'{disease}_{metric}_{column}_histogram.png')
        plt.savefig(output_path)
        plt.close()

In [ ]:
# output_path = 'RESULTS/DISTRIBUTION_RESULTS/distance_metrics_results.csv'
# distance_metrics_results = pd.read_csv(output_path)

# average_results = distance_metrics_results.groupby(['disease', 'metric']).mean(numeric_only=True).reset_index()
# average_results
# highest_lowest_metrics = average_results.groupby('disease').agg(
#     highest_bhattacharyya=('bhattacharyya_distance', lambda x: average_results.loc[x.idxmax(), 'metric']),
#     lowest_bhattacharyya=('bhattacharyya_distance', lambda x: average_results.loc[x.idxmin(), 'metric']),
#     highest_kl=('kl_divergence', lambda x: average_results.loc[x.idxmax(), 'metric']),
#     lowest_kl=('kl_divergence', lambda x: average_results.loc[x.idxmin(), 'metric']),
#     highest_euclidean=('euclidean_distance', lambda x: average_results.loc[x.idxmax(), 'metric']),
#     lowest_euclidean=('euclidean_distance', lambda x: average_results.loc[x.idxmin(), 'metric']),
#     highest_mahalanobis=('mahalanobis_distance', lambda x: average_results.loc[x.idxmax(), 'metric']),
#     lowest_mahalanobis=('mahalanobis_distance', lambda x: average_results.loc[x.idxmin(), 'metric'])
# ).reset_index()

# highest_lowest_metrics

In [ ]:
# output_path = 'RESULTS/DISTRIBUTION_RESULTS/distance_metrics_results.csv'
# distance_metrics_results = pd.read_csv(output_path)

# # Find the bundle/metric with the highest and lowest value for each distance metric for each disease
# highest_lowest_values = distance_metrics_results.groupby('disease').agg(
#     highest_bhattacharyya=('bhattacharyya_distance', lambda x: distance_metrics_results.loc[x.idxmax(), 'metric_bundle']),
#     lowest_bhattacharyya=('bhattacharyya_distance', lambda x: distance_metrics_results.loc[x.idxmin(), 'metric_bundle']),
#     highest_kl=('kl_divergence', lambda x: distance_metrics_results.loc[x.idxmax(), 'metric_bundle']),
#     lowest_kl=('kl_divergence', lambda x: distance_metrics_results.loc[x.idxmin(), 'metric_bundle']),
#     highest_euclidean=('euclidean_distance', lambda x: distance_metrics_results.loc[x.idxmax(), 'metric_bundle']),
#     lowest_euclidean=('euclidean_distance', lambda x: distance_metrics_results.loc[x.idxmin(), 'metric_bundle']),
#     highest_mahalanobis=('mahalanobis_distance', lambda x: distance_metrics_results.loc[x.idxmax(), 'metric_bundle']),
#     lowest_mahalanobis=('mahalanobis_distance', lambda x: distance_metrics_results.loc[x.idxmin(), 'metric_bundle'])
# ).reset_index()

# highest_lowest_values

In [4]:
output_path = f'{MAINFOLDER}/distance_metrics_results.csv'
results_df = pd.read_csv(output_path)
# Create a DataFrame to store the results
top_3_smallest_d_cohen = []

# Iterate over each disease
for disease in results_df['disease'].unique():
    disease_subset = results_df[results_df['disease'] == disease]
    
    # Iterate over each metric
    for metric in disease_subset['metric'].unique():
        metric_subset = disease_subset[disease_subset['metric'] == metric]
        
        # Sort by d_cohen and select the top 3 smallest values
        smallest_d_cohen = metric_subset.nsmallest(3, 'd_cohen')
        
        # Add the results to the list
        for _, row in smallest_d_cohen.iterrows():
            top_3_smallest_d_cohen.append({
                'disease': disease,
                'metric': metric,
                'bundle': row['metric_bundle'],
                'd_cohen': row['d_cohen']
            })

# Convert the results into a DataFrame
top_3_smallest_d_cohen_df = pd.DataFrame(top_3_smallest_d_cohen)

# Display the results
print(top_3_smallest_d_cohen_df)

    disease metric                       bundle   d_cohen
0        AD    adt               adt_mni_IFOF_R  0.191462
1        AD    adt  adt_mni_IIT_mask_skeletonFA  0.296575
2        AD    adt                adt_mni_ICP_R  0.312118
3        AD     rd                 rd_mni_ICP_R  0.200502
4        AD     rd                 rd_mni_ICP_L  0.248057
..      ...    ...                          ...       ...
295  SYN_-1     fw                   fw_mni_MCP  1.000150
296  SYN_-1     fw                  fw_mni_UF_L  1.000165
297  SYN_-1    fat                fat_mni_STT_L  1.000055
298  SYN_-1    fat                fat_mni_ICP_L  1.000060
299  SYN_-1    fat                  fat_mni_MCP  1.000061

[300 rows x 4 columns]


In [6]:
# 1. Enlève les maladies SYN
filtered_df = results_df[~results_df['disease'].str.startswith('SYN')]

# 2. Moyenne de d_cohen pour chaque combinaison metric + bundle
mean_dcohen = (
    filtered_df
    .groupby(['metric', 'bundle'])['d_cohen']
    .mean()
    .reset_index()
)

# 3. Top 3 des bundles avec la plus petite moyenne de d_cohen pour chaque metric
top3_per_metric = (
    mean_dcohen
    .sort_values(['metric', 'd_cohen'])
    .groupby('metric')
    .head(3)
    .reset_index(drop=True)
)

# Affichage
print(top3_per_metric)


   metric     bundle   d_cohen
0      ad  mni_ICP_R  0.138507
1      ad  mni_ICP_L  0.143599
2      ad    mni_MCP  0.190170
3     adt  mni_ICP_R  0.140043
4     adt  mni_ICP_L  0.146304
5     adt    mni_MCP  0.200017
6     afd  mni_F_L_R  0.197745
7     afd     mni_AC  0.213232
8     afd   mni_OR_R  0.213256
9      fa   mni_ML_R  0.127405
10     fa  mni_STT_R  0.130809
11     fa  mni_STT_L  0.136513
12    fat  mni_ICP_L  0.174757
13    fat  mni_ICP_R  0.179768
14    fat    mni_MCP  0.204329
15     fw  mni_ICP_L  0.229651
16     fw    mni_MCP  0.230657
17     fw  mni_AST_R  0.240591
18     md  mni_ICP_R  0.146979
19     md  mni_ICP_L  0.151578
20     md    mni_MCP  0.161687
21    mdt  mni_ICP_R  0.144038
22    mdt  mni_ICP_L  0.148043
23    mdt  mni_CST_R  0.150722
24     rd    mni_MCP  0.137880
25     rd  mni_ICP_R  0.140330
26     rd  mni_CST_R  0.145597
27    rdt  mni_ICP_R  0.125416
28    rdt  mni_ICP_L  0.136099
29    rdt    mni_MCP  0.136748


In [7]:
# 1. Toujours filtrer les maladies SYN
filtered_df = results_df[~results_df['disease'].str.startswith('SYN')]

# 2. Moyenne de d_cohen pour chaque (metric, bundle)
mean_dcohen_metric_bundle = (
    filtered_df
    .groupby(['metric', 'bundle'])['d_cohen']
    .mean()
    .reset_index()
)

# 3. Moyenne de d_cohen pour chaque bundle (tous metrics confondus)
mean_dcohen_per_bundle = (
    mean_dcohen_metric_bundle
    .groupby('bundle')['d_cohen']
    .mean()
    .reset_index()
)

# 4. Prendre les 3 bundles avec la plus petite moyenne overall
top3_bundles_overall = mean_dcohen_per_bundle.nsmallest(3, 'd_cohen')

# Afficher le résultat
print(top3_bundles_overall)


       bundle   d_cohen
17  mni_ICP_R  0.164943
16  mni_ICP_L  0.168220
23    mni_MCP  0.185897


In [8]:
# 1. Reprend les 3 meilleurs bundles (si pas déjà dans la variable)
top_bundles = top3_bundles_overall['bundle'].tolist()

# 2. Filtrer les lignes associées à ces bundles (et exclure SYN)
worst_cases = results_df[
    (results_df['bundle'].isin(top_bundles)) &
    (~results_df['disease'].str.startswith('SYN'))
]

# 3. Trouver le pire cas pour chaque bundle
worst_per_bundle = (
    worst_cases.loc[
        worst_cases.groupby('bundle')['d_cohen'].idxmax()
    ][['bundle', 'disease', 'metric', 'd_cohen']]
    .reset_index(drop=True)
)

# Afficher le résultat
print(worst_per_bundle)


      bundle disease metric   d_cohen
0  mni_ICP_L    ADHD    afd  0.515280
1  mni_ICP_R    ADHD    afd  0.500354
2    mni_MCP    ADHD    afd  0.576823
